<a href="https://colab.research.google.com/github/sjana01/m2pi2021/blob/main/adult_census.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd

In [ ]:
text = requests.get("https://raw.githubusercontent.com/mgreenbe/adult-census/master/adult.names").text
names_and_descrs = [line.split(":") for line in text.strip().split("\n")[-14:]]
names_and_descrs = [(name, descr[:-1].strip()) for name, descr in names_and_descrs]
names = [name for name, desc in names_and_descrs]
continuous = [name for (name, descr) in names_and_descrs if descr == "continuous"]
categorical = {name: descr.split(", ") for (name, descr) in names_and_descrs if name not in continuous}
categorical["y"] = ["<=50K", ">50K"]
print(continuous)
print(categorical)

['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
{'workclass': ['Private', 'Self-emp-not-inc', 'Self-emp-inc', 'Federal-gov', 'Local-gov', 'State-gov', 'Without-pay', 'Never-worked'], 'education': ['Bachelors', 'Some-college', '11th', 'HS-grad', 'Prof-school', 'Assoc-acdm', 'Assoc-voc', '9th', '7th-8th', '12th', 'Masters', '1st-4th', '10th', 'Doctorate', '5th-6th', 'Preschool'], 'marital-status': ['Married-civ-spouse', 'Divorced', 'Never-married', 'Separated', 'Widowed', 'Married-spouse-absent', 'Married-AF-spouse'], 'occupation': ['Tech-support', 'Craft-repair', 'Other-service', 'Sales', 'Exec-managerial', 'Prof-specialty', 'Handlers-cleaners', 'Machine-op-inspct', 'Adm-clerical', 'Farming-fishing', 'Transport-moving', 'Priv-house-serv', 'Protective-serv', 'Armed-Forces'], 'relationship': ['Wife', 'Own-child', 'Husband', 'Not-in-family', 'Other-relative', 'Unmarried'], 'race': ['White', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other', 'Black'], 

In [ ]:
df_train = pd.read_csv("https://raw.githubusercontent.com/mgreenbe/adult-census/master/adult.data",
                 index_col=False,
                 names=[*names, "y"],
                 dtype={name: "category" for name in categorical},
                 sep=", ",
                 engine="python")
df_train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,y
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [ ]:
df_test = pd.read_csv("https://raw.githubusercontent.com/mgreenbe/adult-census/master/adult.test",
                 index_col=False,
                 names=[*names, "y"],
                #  dtype={name: "category" for name in categorical},
                 sep=", ",
                 skiprows=1,
                 engine="python")
df_test.y = df_test.y.apply(lambda s: s[:-1]) # there's a period at the end of each row of the test file!
df_test.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,y
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [ ]:
df = pd.concat([df_train, df_test], axis=0).reset_index(drop=True)
y = df.y.values
df = df.drop(columns=["y"])
df[continuous] = (df[continuous] - df_train[continuous].mean(axis=0))/df_train[continuous].std(axis=0)
df = pd.get_dummies(df)
X = df.values


X_train = X[:len(df_train)]
y_train = y[:len(df_train)]
X_test = X[len(df_train):]
y_test = y[len(df_train):]
assert len(X_test) == len(df_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2)
model = LogisticRegression(max_iter=1000)
model.fit(X_tr, y_tr)
y_hat = model.predict(X_val)
accuracy_score(y_test, y_hat)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=500)
model.fit(X_train, y_train)
y_hat = model.predict(X_test)
accuracy_score(y_test, y_hat)

0.8522203795835637